# sentencepiece 사용

## do it -> fasttext(페이스북), BERT(구글), GloVe 패키지 사용
## model ensemble

In [ ]:
pip install fasttext

In [ ]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv('test_x.csv')

In [ ]:
pd.options.display.max_rows = 2000

In [ ]:
display(train,test)

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


In [ ]:
import pandas as pd
import numpy as np
import re

train = pd.read_csv("train.csv")
test = pd.read_csv('test_x.csv')

"""
# 필요없는 단어 제거
def alpha_num(text):
  return re.sub(r'[^a-zA-z0-9\s]', '', text)

# 소문자로 변환
train['text'] = train['text'].str.lower().apply(alpha_num)
test['text'] = test['text'].str.lower().apply(alpha_num)

re_list = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+') # 특수문자,기호단어 제거 리스트
def removeing(text):
  re_html= re_list.sub('',text)
  return re_html
# 제거 실행
train['text']=train['text'].str.lower().apply(lambda x : removeing(x))
test['text']=test['text'].str.lower().apply(lambda x : removeing(x))
"""

file = open('fasttexttrain.txt','w+')
for i in train.index:
    line = '__label__' + str(train['author'][i])+' '+train['text'][i]
    file.write(line+"\n")

In [ ]:
import fasttext
text_clf_model = fasttext.train_supervised('fasttexttrain.txt', epoch=77, minCount=2, maxn=52, verbose=0, lrUpdateRate=42, neg=3)
print(text_clf_model.words)
print(text_clf_model.labels)
print("end")

submission = pd.read_csv('sample_submission.csv', index_col=False)

# Skipgram model : 랜덤한 인접단어를 사용해서 예측
#model = fasttext.train_unsupervised('data.txt', model='skipgram')
#print(model.words)

# or, cbow model : 단어들의 벡터를 합계로 사용해서 예측
#model = fasttext.train_unsupervised('data.txt', model='cbow')
#print(model.words)

['the', 'and', 'to', 'of', '</s>', 'a', 'I', 'in', 'was', 'that', 'he', 'his', 'you', 'with', 'had', 'it', 'odin', 'for', 'as', 'at', 'not', 'my', 'have', 'is', 'be', 'her', 'said', 'on', 'she', 'but', 'odin,', 'all', 'from', 'me', 'by', 'him', 'were', 'which', 'so', 'this', 'been', 'would', 'The', 'an', 'your', 'are', 'very', 'no', 'He', 'one', 'Mr.', 'could', 'what', 'we', 'they', 'will', 'if', 'or', 'upon', 'there', 'when', 'out', 'do', 'who', 'up', '“I', 'some', 'am', 'odin.', 'more', 'into', 'It', 'their', 'about', 'But', 'only', 'should', 'little', 'must', 'than', 'like', 'any', 'know', 'such', 'did', 'man', 'has', 'see', 'And', 'them', 'can', 'our', 'never', 'much', 'come', 'before', 'She', 'down', 'may', 'think', 'how', 'made', 'it,', 'time', 'me,', 'came', 'now', 'great', 'two', 'good', 'might', 'old', 'You', 'over', 'shall', 'after', 'own', 'cried', 'though', 'him,', 'go', 'it.', 'say', 'without', 'even', 'other', 'you,', 'looked', 'Mrs.', 'then', 'Miss', 'thought', 'every', 

# baseline
## (epoch = 56, minCount = 2, max = 22, verbose = 0) - 0.30907

## epoch=72, minCount=2, maxn=52, verbose=0 - 0.30622


In [ ]:
# 모델의 크기를 줄이고 Memory Footprint를 축소합니다
#text_clf_model.quantize()

In [ ]:
for i in test.index:
    lable, proba = text_clf_model.predict(test['text'][i], k=5)
    for la, pr in zip(lable, proba):
        if '__label__0' == la:
            submission.loc[i, '0'] = pr
        elif '__label__1' == la:
            submission.loc[i, '1'] = pr
        elif '__label__2' == la:
            submission.loc[i, '2'] = pr
        elif '__label__3' == la:
            submission.loc[i, '3'] = pr
        elif '__label__4' == la:
            submission.loc[i, '4'] = pr
    # submission.loc[i, '0'] = proba[lable.loc('__label__0')]
    # submission.loc[i, '1'] = proba[4]
    # submission.loc[i, '2'] = proba[2]
    # submission.loc[i, '3'] = proba[0]
    # submission.loc[i, '4'] = proba[3]

submission.to_csv('lastms_submission.csv', index=False)
print('end')


end


In [ ]:
submission

,index,0,1,2,3,4
0,0,0.007042,0.584447,0.255571,0.152706,0.000285
1,1,0.054392,0.943620,0.000050,0.000719,0.001270
2,2,0.991976,0.005144,0.000360,0.001409,0.001161
3,3,0.000694,0.000026,0.993504,0.000107,0.005719
4,4,0.819438,0.011397,0.003822,0.158539,0.006853
...,...,...,...,...,...,...
19612,19612,0.003791,0.995963,0.000024,0.000220,0.000052
19613,19613,0.099009,0.000374,0.007665,0.000781,0.892221
19614,19614,0.000882,0.997404,0.000013,0.001733,0.000017
19615,19615,0.004182,0.987553,0.007714,0.000502,0.000100


In [ ]:
submission

,index,0,1,2,3,4
0,0,0.007042,0.584447,0.255571,0.152706,0.000285
1,1,0.054392,0.943620,0.000050,0.000719,0.001270
2,2,0.991976,0.005144,0.000360,0.001409,0.001161
3,3,0.000694,0.000026,0.993504,0.000107,0.005719
4,4,0.819438,0.011397,0.003822,0.158539,0.006853
...,...,...,...,...,...,...
19612,19612,0.003791,0.995963,0.000024,0.000220,0.000052
19613,19613,0.099009,0.000374,0.007665,0.000781,0.892221
19614,19614,0.000882,0.997404,0.000013,0.001733,0.000017
19615,19615,0.004182,0.987553,0.007714,0.000502,0.000100


In [ ]:
facebooksub = submissionf

In [ ]:
facebooksub

,index,0,1,2,3,4
0,0,0.007584,0.579728,0.252649,0.159783,0.000306
1,1,0.053731,0.944216,0.000049,0.000766,0.001288
2,2,0.992039,0.004938,0.000382,0.001451,0.001240
3,3,0.000708,0.000025,0.993260,0.000108,0.005949
4,4,0.818699,0.011880,0.003912,0.158598,0.006960
...,...,...,...,...,...,...
19612,19612,0.003588,0.996184,0.000022,0.000206,0.000049
19613,19613,0.097186,0.000361,0.007580,0.000751,0.894171
19614,19614,0.000915,0.997319,0.000013,0.001786,0.000017
19615,19615,0.004319,0.986557,0.008542,0.000522,0.000110


# AUTO ML 모델


In [ ]:
!pip install autokeras

Processing /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6/keras_tuner-1.0.2-cp36-none-any.whl
  Found existing installation: keras-tuner 1.0.2rc4
    Uninstalling keras-tuner-1.0.2rc4:
      Successfully uninstalled keras-tuner-1.0.2rc4


In [ ]:
#!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.

  Cloning https://github.com/keras-team/keras-tuner-1.0.2rc0 to /tmp/pip-req-build-pym1vrlr
  Running command git clone -q https://github.com/keras-team/keras-tuner-1.0.2rc0 /tmp/pip-req-build-pym1vrlr
  fatal: could not read Username for 'https://github.com': No such device or address
ERROR: Command errored out with exit status 128: git clone -q https://github.com/keras-team/keras-tuner-1.0.2rc0 /tmp/pip-req-build-pym1vrlr Check the logs for full command output.


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import autokeras as ak

ImportError: ignored

In [ ]:
auto_train = pd.read_csv("train.csv")
auto_X = auto_train.text.values
auto_Y =  auto_train['author'].values

In [ ]:
input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.ClassificationHead()(output_node)
auto_clf = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    max_trials=5
    )
auto_clf.fit(auto_X, auto_Y, validation_split = 0.1, epochs=5)

Trial 5 Complete [00h 03m 35s]
val_loss: 0.7223118543624878

Best val_loss So Far: 0.6986198425292969
Total elapsed time: 00h 34m 15s
INFO:tensorflow:Oracle triggered exit
Epoch 1/5
1715/1715 [==============================] - 44s 26ms/step - loss: 1.1484 - accuracy: 0.5162
Epoch 2/5
1715/1715 [==============================] - 45s 26ms/step - loss: 0.6101 - accuracy: 0.7766
Epoch 3/5
1715/1715 [==============================] - 45s 26ms/step - loss: 0.4246 - accuracy: 0.8475
Epoch 4/5
1715/1715 [==============================] - 44s 26ms/step - loss: 0.3251 - accuracy: 0.8852
Epoch 5/5
1715/1715 [==============================] - 44s 26ms/step - loss: 0.2596 - accuracy: 0.9085
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [ ]:
auto_model = auto_clf.export_model()
me = auto_clf.evaluate(auto_X,auto_Y)

1715/1715 [==============================] - 7s 4ms/step - loss: 0.1774 - accuracy: 0.9403


In [ ]:
me

[0.17735262215137482, 0.9403414726257324]

In [ ]:
auto_test = pd.read_csv("test_x.csv")
auto_submission = pd.read_csv("sample_submission.csv")
#auto_pred = auto_model.predict(test.text.values)
#auto_pred
result = auto_model.predict(auto_test.text.values,use_multiprocessing=True)

In [ ]:
result

array([[1.6863106e-02, 9.7965962e-01, 9.9744555e-04, 2.4687098e-03,
        1.1137904e-05],
       [4.0575886e-01, 5.8399588e-01, 1.5955601e-03, 2.2197268e-03,
        6.4299735e-03],
       [1.0000000e+00, 1.4948270e-10, 4.0476624e-12, 2.6689161e-11,
        3.9251930e-13],
       ...,
       [3.7032240e-08, 1.0000000e+00, 2.0970486e-17, 2.1387961e-10,
        1.3917716e-12],
       [3.3021152e-03, 9.9669719e-01, 2.2609166e-07, 2.9597285e-07,
        2.2393635e-07],
       [9.9174249e-01, 5.7718583e-11, 5.0575146e-04, 1.1801909e-06,
        7.7506294e-03]], dtype=float32)

In [ ]:
#auto_pred
result

array([[1.6863106e-02, 9.7965962e-01, 9.9744555e-04, 2.4687098e-03,
        1.1137904e-05],
       [4.0575886e-01, 5.8399588e-01, 1.5955601e-03, 2.2197268e-03,
        6.4299735e-03],
       [1.0000000e+00, 1.4948270e-10, 4.0476624e-12, 2.6689161e-11,
        3.9251930e-13],
       ...,
       [3.7032240e-08, 1.0000000e+00, 2.0970486e-17, 2.1387961e-10,
        1.3917716e-12],
       [3.3021152e-03, 9.9669719e-01, 2.2609166e-07, 2.9597285e-07,
        2.2393635e-07],
       [9.9174249e-01, 5.7718583e-11, 5.0575146e-04, 1.1801909e-06,
        7.7506294e-03]], dtype=float32)

In [ ]:
"""
for a in range(0,19616):
  for j in range(0,5):
    auto_pred_split = result[a][j]/sum(result[a])
    my_list = auto_pred_split[a]
  print(my_list)
  auto_submission[['0','1','2','3','4']] =  my_list[a]
  """

"\nfor a in range(0,19616):\n  for j in range(0,5):\n    auto_pred_split = result[a][j]/sum(result[a])\n    my_list = auto_pred_split[a]\n  print(my_list)\n  auto_submission[['0','1','2','3','4']] =  my_list[a]\n  "

In [ ]:
#auto_submission[['0','1','2','3','4']] = auto_pred
#auto_submission[['0','1','2','3','4']] = auto_pred_proba
#auto_submission = auto_submission.drop("author",axis=1)

In [ ]:
#auto_submission.to_csv("automl_proba_submission.csv", index = False)
#auto_submission

,index,0,1,2,3,4
0,0,9.542074e-03,6.481229e-02,7.206826e-02,8.535725e-01,4.989331e-06
1,1,8.338007e-01,1.607351e-01,2.999921e-03,1.691555e-03,7.726785e-04
2,2,1.000000e+00,8.153820e-10,1.091636e-14,9.986077e-14,1.494363e-10
3,3,2.136642e-04,1.504646e-10,9.733175e-01,9.546341e-08,2.646880e-02
4,4,9.972480e-01,1.642083e-09,3.843228e-06,2.731874e-03,1.623596e-05
...,...,...,...,...,...,...
19612,19612,1.700805e-04,9.998298e-01,1.614291e-13,1.048525e-07,2.422057e-10
19613,19613,9.014540e-01,5.054074e-13,3.587805e-05,4.852191e-10,9.851009e-02
19614,19614,2.267865e-07,9.999998e-01,3.163272e-16,1.769430e-11,1.132108e-13
19615,19615,2.849217e-04,9.997136e-01,1.514467e-08,1.404485e-06,9.881125e-11


# Sentencepiece

In [ ]:
pip install sentencepiece -i https://pypi.python.org/simple

Looking in indexes: https://pypi.python.org/simple
     |████████████████████████████████| 1.1MB 5.8MB/s 


In [ ]:
# nltk
import nltk  
nltk.download('stopwords') 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from collections import Counter
import cufflinks as cf
cf.go_offline()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

In [ ]:
from pandas import read_csv
import re
from sentencepiece import SentencePieceTrainer,SentencePieceProcessor
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import  EarlyStopping
from tensorflow.python.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from keras.models import load_model
from numpy import mean
from pandas import DataFrame
from random import randint

# Data Loading
## 1. file에서 data 불러오기
## 2. Sentence Piece로 모델 학습
## 3. CountVectorizer에 tokenizer로 trained model 사용
## 4. 학습용 데이터 최종 Loading

In [ ]:

# data라는 폴더 안에 데이터를 저장해 두었습니다.
train = read_csv('train.csv', engine='python')
test = read_csv('test_x.csv', engine='python')
y_train = train['author'].values
print('data load completed')

  # 필요없는 단어 제거
def alpha_num(text):
    return re.sub(r'[^a-zA-z0-9\s]', '', text)

  # 소문자로 변환
train['text'] = train['text'].str.lower().apply(alpha_num)
test['text'] = test['text'].str.lower().apply(alpha_num)

re_list = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+') # 특수문자,기호단어 제거 리스트
def removeing(text):
    re_html= re_list.sub('',text)
    return re_html
    # 제거 실행
train['text']=train['text'].apply(lambda x : removeing(x))
test['text']=test['text'].apply(lambda x : removeing(x))

  # SentencePieceTrainer로 학습할 데이터 준비
with open('author.txt','w',encoding='utf-8') as f:
    f.write('\n'.join(train['text']))

  # 3000자까지 vocab size를 지정해서 학습
SentencePieceTrainer.Train('--input=author.txt --model_prefix=author --vocab_size=3000')
  
sp = SentencePieceProcessor()
sp.Load("author.model")

  # 학습된 모델을 tokenizer로 사용 / min_df = 3은 최소 3번 이상 나타난 단어를 의미합니다
cv = CountVectorizer(lowercase = False, tokenizer = sp.encode_as_pieces, min_df = 3)

  # 최종적으로 사용할 학습데이터입니다.
tdm_train = cv.fit_transform(train['text']).toarray()
tdm_test = cv.transform(test['text']).toarray()
print('data setting completed')


data load completed
data setting completed


# 패딩 Padding

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def create_corpus(df):
    corpus=[]
    for tweet in tqdm(df['text']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

In [ ]:
corpus=create_corpus(train)
corpus=create_corpus(test)

100%|██████████| 19617/19617 [00:07<00:00, 2701.31it/s]


In [ ]:
corpus

[['think',
  'one',
  'charming',
  'young',
  'ladies',
  'ever',
  'met',
  'might',
  'useful',
  'work',
  'decided',
  'genius',
  'way',
  'witness',
  'way',
  'preserved',
  'agra',
  'plan',
  'papers',
  'father',
  'love',
  'emotional',
  'thing',
  'whatever',
  'emotional',
  'opposed',
  'true',
  'cold',
  'reason',
  'place',
  'things',
  'never',
  'marry',
  'lest',
  'bias',
  'judgment'],
 ['replied',
  'sudden',
  'consciousness',
  'find',
  'ignorant',
  'goodness',
  'owe',
  'alli',
  'feel',
  'iti',
  'would',
  'express',
  'couldbut',
  'well',
  'know',
  'orator'],
 ['lady',
  'stated',
  'intention',
  'screaming',
  'course',
  'would',
  'screamed',
  'additional',
  'boldness',
  'exertion',
  'rendered',
  'unnecessary',
  'hasty',
  'knocking',
  'door',
  'sooner',
  'heard',
  'mr',
  'odin',
  'darted',
  'much',
  'agility',
  'wine',
  'bottles',
  'began',
  'dusting',
  'great',
  'violence',
  'matron',
  'sharply',
  'demanded'],
 ['sudde

In [ ]:
MAX_LEN=50
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [ ]:
tweet_pad

array([[  32,    3, 1337, ...,    0,    0,    0],
       [ 303,  434, 1167, ...,    0,    0,    0],
       [ 117, 3039, 1338, ...,    0,    0,    0],
       ...,
       [   1,  362, 1863, ...,    0,    0,    0],
       [  88,    6,   17, ...,    0,    0,    0],
       [ 514, 6514,    8, ...,  796,  301,  424]], dtype=int32)

In [ ]:
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

Number of unique words: 42403


In [ ]:
word_index

{'odin': 1,
 'would': 2,
 'one': 3,
 'could': 4,
 'said': 5,
 'upon': 6,
 'mr': 7,
 'man': 8,
 'time': 9,
 'little': 10,
 'know': 11,
 'like': 12,
 'must': 13,
 'see': 14,
 'well': 15,
 'much': 16,
 'odins': 17,
 'come': 18,
 'never': 19,
 'two': 20,
 'though': 21,
 'us': 22,
 'might': 23,
 'good': 24,
 'say': 25,
 'made': 26,
 'great': 27,
 'may': 28,
 'even': 29,
 'first': 30,
 'way': 31,
 'think': 32,
 'every': 33,
 'old': 34,
 'came': 35,
 'nothing': 36,
 'day': 37,
 'go': 38,
 'long': 39,
 'thought': 40,
 'last': 41,
 'back': 42,
 'mrs': 43,
 'hand': 44,
 'without': 45,
 'away': 46,
 'house': 47,
 'still': 48,
 'face': 49,
 'shall': 50,
 'went': 51,
 'eyes': 52,
 'make': 53,
 'sir': 54,
 'yet': 55,
 'looked': 56,
 'dont': 57,
 'room': 58,
 'quite': 59,
 'ever': 60,
 'young': 61,
 'take': 62,
 'life': 63,
 'saw': 64,
 'something': 65,
 'miss': 66,
 'mind': 67,
 'tell': 68,
 'seemed': 69,
 'look': 70,
 'left': 71,
 'head': 72,
 'always': 73,
 'door': 74,
 'night': 75,
 'another': 76

In [ ]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))

In [ ]:
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
train=tweet_pad[:train.shape[0]]
test=tweet_pad[train.shape[0]:]

In [ ]:
train

array([[  32,    3, 1337, ...,    0,    0,    0],
       [ 303,  434, 1167, ...,    0,    0,    0],
       [ 117, 3039, 1338, ...,    0,    0,    0],
       ...,
       [   1,  362, 1863, ...,    0,    0,    0],
       [  88,    6,   17, ...,    0,    0,    0],
       [ 514, 6514,    8, ...,  796,  301,  424]], dtype=int32)

# Simple DNN Code
## Input Shape는 vocabsize = 3000을 기준으로 약 3000개의 단어가 들어갑니다.

In [ ]:
# 간단한 DNN 모델
def create_dnn(input_shape):
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape = (input_shape,)))
    model.add(Dropout(0.15))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dropout(0.15))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dropout(0.3))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = Adam(), metrics=['accuracy'])
    return model


# Creating CSV
## 1. StratifiedKFold로 5 fold
## 2. val_loss를 기준으로 Early Stopping &  Model Checkpoint monitoring
## 3. Checked Model 불러와 tdm_test 예측 + predicts에 저장
## 4. log_loss로 Validation data score 저장
## 5. 5 Fold의 predicts의 mean => csv로 저장

In [ ]:
batch_size =  256
epochs = 32
seed = randint(1,1000000)

  # SKF 중 생성되는 결과물 저장 
predicts = []
  # Validation Score 저장
scores = []
  
  
  # 과적합이 쉽게 되어 Patience는 2
es = EarlyStopping(monitor='val_loss', verbose=0, patience=2)

  # 학습하며 생되는 model 저장
filepath_val_loss="/content/drive/MyDrive/Dacon-소설작가분류/models/best_model_cdnn.tf"
checkpoint_val_loss = ModelCheckpoint(filepath_val_loss, monitor='val_loss', verbose=0, save_best_only=True, mode='min')

  # SKF
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for train_index, val_index in skf.split(tdm_train, y_train):
  model = create_dnn(tdm_train.shape[1])
  tr_X = tdm_train[train_index]
  tr_y = y_train[train_index]

  val_X = tdm_train[val_index]
  val_y = y_train[val_index]

  model.fit(tr_X,tr_y,
                      batch_size=batch_size,
                      epochs=epochs, 
                      validation_data=(val_X,val_y), 
                      shuffle=True, 
                      verbose=0,
                      callbacks=[es,checkpoint_val_loss]
                      ) 

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Dacon-소설작가분류/models/best_model_cdnn.tf/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Dacon-소설작가분류/models/best_model_cdnn.tf/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Dacon-소설작가분류/models/best_model_cdnn.tf/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Dacon-소설작가분류/models/best_model_cdnn.tf/assets


# 모델 평가셋
- 0.7050569653511047
- seed5,max 0.7063325643539429

In [ ]:
results = model.evaluate(val_X, val_y)
print('Test accuracy: ', results[1])

343/343 [==============================] - 1s 2ms/step - loss: 0.8345 - accuracy: 0.7226
Test accuracy:  0.7225512266159058


In [ ]:
    
model = load_model(filepath_val_loss)
predicts.append(model.predict_proba(tdm_test))
score = log_loss(val_y,model.predict_proba(val_X))
scores.append(score)
del tr_X,tr_y,val_X,val_y,model,score
  
# predicts에 저장된 predicts 저장
prediction = DataFrame(mean(predicts,axis=0)).reset_index()

Instructions for updating:
Please use `model.predict()` instead.


In [ ]:
prediction # using the nltk

,index,0,1,2,3,4
0,0,2.648345e-02,0.021173,9.003751e-01,2.839874e-02,2.357006e-02
1,1,1.093634e-01,0.824131,1.376326e-02,3.272033e-02,2.002221e-02
2,2,9.966866e-01,0.002120,2.751543e-04,8.245849e-05,8.356981e-04
3,3,1.199710e-05,0.000001,9.999076e-01,7.470852e-06,7.149098e-05
4,4,9.253692e-01,0.034474,1.432715e-02,1.649001e-02,9.339345e-03
...,...,...,...,...,...,...
19612,19612,2.004391e-09,1.000000,4.925350e-14,1.750284e-11,1.353987e-18
19613,19613,3.707558e-03,0.000034,1.374854e-03,1.455105e-04,9.947384e-01
19614,19614,3.451563e-03,0.996340,5.932309e-05,1.476114e-04,1.229151e-06
19615,19615,9.175776e-03,0.984329,1.379671e-03,4.903684e-03,2.119013e-04


In [ ]:
import pandas as pd
sub = pd.read_csv("sample_submission.csv")

In [ ]:
sub = prediction

In [ ]:
sub

,index,0,1,2,3,4
0,0,2.648345e-02,0.021173,9.003751e-01,2.839874e-02,2.357006e-02
1,1,1.093634e-01,0.824131,1.376326e-02,3.272033e-02,2.002221e-02
2,2,9.966866e-01,0.002120,2.751543e-04,8.245849e-05,8.356981e-04
3,3,1.199710e-05,0.000001,9.999076e-01,7.470852e-06,7.149098e-05
4,4,9.253692e-01,0.034474,1.432715e-02,1.649001e-02,9.339345e-03
...,...,...,...,...,...,...
19612,19612,2.004391e-09,1.000000,4.925350e-14,1.750284e-11,1.353987e-18
19613,19613,3.707558e-03,0.000034,1.374854e-03,1.455105e-04,9.947384e-01
19614,19614,3.451563e-03,0.996340,5.932309e-05,1.476114e-04,1.229151e-06
19615,19615,9.175776e-03,0.984329,1.379671e-03,4.903684e-03,2.119013e-04


In [ ]:
sub.to_csv("dnnper_submission.csv", index=False, header=True)

In [ ]:
sub

,index,0,1,2,3,4
0,0,2.648345e-02,0.021173,9.003751e-01,2.839874e-02,2.357006e-02
1,1,1.093634e-01,0.824131,1.376326e-02,3.272033e-02,2.002221e-02
2,2,9.966866e-01,0.002120,2.751543e-04,8.245849e-05,8.356981e-04
3,3,1.199710e-05,0.000001,9.999076e-01,7.470852e-06,7.149098e-05
4,4,9.253692e-01,0.034474,1.432715e-02,1.649001e-02,9.339345e-03
...,...,...,...,...,...,...
19612,19612,2.004391e-09,1.000000,4.925350e-14,1.750284e-11,1.353987e-18
19613,19613,3.707558e-03,0.000034,1.374854e-03,1.455105e-04,9.947384e-01
19614,19614,3.451563e-03,0.996340,5.932309e-05,1.476114e-04,1.229151e-06
19615,19615,9.175776e-03,0.984329,1.379671e-03,4.903684e-03,2.119013e-04
